In [ ]:
!pwd

import sys
import os

# Detect if we are in Colab or a generic cloud environment (like Vast.ai)
is_colab = 'google.colab' in sys.modules 
is_vast = os.path.exists('/root/.vast_container') # Common Vast.ai indicator
is_cloud = is_colab or is_vast

if is_cloud:
  py_url = "https://raw.githubusercontent.com/fanaustinca/so101ai/refs/heads/main/lerobot_setup.py"

  !curl -O {py_url}
  if os.path.exists("lerobot_setup.py"):
    print("\nSuccessfully downloaded lerobot_setup.py")
  else:
    print("\nDownload failed.")
else:
  print("Running on local machine. Skipping cloud setup.")

In [ ]:
%conda install python-dotenv -y

# Setting lerobot env
from lerobot_setup import setup_lerobot_env

setup_lerobot_env()


In [ ]:
import json
import sys

from IPython.display import display, Markdown

from lerobot_setup import cd_lerobot

cd_lerobot()

# Training ACT model.
# Adjust the following variables before starting training
policy_type = 'diffusion'
hf_namespace = 'fanaustinca'
model_repo = f"{hf_namespace}/fll_diffusion_model_400episodes_v2"
eval_repo = f"{hf_namespace}/fll_eval_act_400episodes_v2"
robot_port = '/dev/ttyACM1'
camera_wrist_index = 2
camera_top_index = 4
num_episodes = 10

camera_config = {
  "wrist": {
    "type": "opencv", 
    "index_or_path": camera_wrist_index, 
    "fps": 30, 
    "width": 640, 
    "height": 480
  },
  "top": {
    "type": "opencv", 
    "index_or_path": camera_top_index, 
    "fps": 30, 
    "width": 640, 
    "height": 480
  }
}
camera_string = json.dumps(camera_config, separators=(',', ':'))

arguments = f"""\
  --robot.type=so101_follower \
  --robot.port={robot_port} \
  --robot.cameras='{camera_string}' \
  --job_name=fll_eval_{policy_type} \
  --policy.device=cuda 
  --policy.use_amp=true \
  --policy.push_to_hub=true \
  --policy.type={policy_type} \
  --policy.path={model_repo} \
  --dataset.repo_id={eval_repo} \
  --dataset.num_episodes={num_episodes} \
  --dataset.single_task="fll_mission" \
  --display_data=true"""

if policy_type == 'diffusion':
    arguments += " --policy.num_inference_steps=30"

command_str = f"{sys.executable} src/lerobot/scripts/lerobot_record.py {arguments}"
display(Markdown(f"```bash\n{command_str}\n```"))
